In [ ]:
import os
from google.cloud import spanner
import random
import string
from google.oauth2 import service_account
import base64
OPERATION_TIMEOUT_SECONDS = 240

class StreamDeFiFutures():

    def __init__(self):
        pass

    @staticmethod
    def ddl():
        ddl = """CREATE TABLE StreamDeFiFutures (
                DATA_KEY STRING(64) NOT NULL,
                FUNDING NUMERIC NOT NULL,
                OPEN_INTEREST NUMERIC NOT NULL,
                ORACLE_PRICE NUMERIC NOT NULL,
                MID_PRICE NUMERIC NOT NULL,
                LONG_FUNDING NUMERIC NOT NULL,
                SHORT_FUNDING NUMERIC NOT NULL,
                APR NUMERIC NOT NULL,
                APY NUMERIC NOT NULL,
                SPREAD NUMERIC NOT NULL,
                SYMBOL     STRING(64) NOT NULL,
                OBSERVATION_TIME TIMESTAMP OPTIONS (allow_commit_timestamp=true)
            ) PRIMARY KEY (DATA_KEY)"""
        return ddl

    def create_database(self):
        instance_id = "defi"
        database_id = "futuresfunding"
        creds = service_account.Credentials.from_service_account_file("key.json")
        spanner_client = spanner.Client(credentials=creds)
        instance = spanner_client.instance(instance_id)

        database = instance.database(
            database_id,
            ddl_statements=[
                StreamDeFiFutures.ddl()
            ],
        )

        operation = database.create()

        print("Waiting for operation to complete...")
        operation.result(OPERATION_TIMEOUT_SECONDS)

        print("Created database {} on instance {}".format(database_id, instance_id))

    @staticmethod
    def insert(instance_dict):
        instance_id = "defi"
        database_id = "futuresfunding"
        creds = service_account.Credentials.from_service_account_file("key.json")
        spanner_client = spanner.Client(credentials=creds)
        instance = spanner_client.instance(instance_id)
        db = instance.database(database_id)


        fields = []
        values = []
        for field in instance_dict:
            fields.append(field)
            values.append(instance_dict[field])
        fieldt = tuple(fields)
        valuest = tuple(values)
        with db.batch() as batch:
            batch.insert(table="StreamDeFiFutures", columns=fieldt,
                         values=[valuest], )

        print("Inserted instance data." + str(instance_dict))

    @staticmethod
    def query():
        instance_id = "defi"
        database_id = "futuresfunding"
        creds = service_account.Credentials.from_service_account_file("key.json")
        spanner_client = spanner.Client(credentials=creds)
        instance = spanner_client.instance(instance_id)
        db = instance.database(database_id)
        with db.snapshot(multi_use=True) as snapshot:
            results = snapshot.execute_sql("SELECT * FROM StreamDeFiFutures ORDER BY OBSERVATION_TIME ASC")
            rows = list(results)
            fields = results.fields
            return_arr = []
            row_counter = 0
            for row in rows:  # for each result
                temp_dict = {}
                field_counter = 0
                for datarow in row:  # for each field
                    temp_dict[fields[field_counter].name] = \
                    rows[row_counter][field_counter]  # construct a dict
                    field_counter = field_counter + 1
                return_arr.append(temp_dict)
                row_counter = row_counter + 1

            aa = return_arr
            return return_arr


stream = StreamDeFiFutures()
stream.create_database()